In [2]:
import tensorflow as tf
from keras.models import model_from_json
import matplotlib.pyplot as plt
import numpy as np
import cv2

Using TensorFlow backend.


In [3]:
json_file = open("finalmodelv2.json","r")
model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(model_json)
loaded_model.load_weights("finalmodelv2.h5")

In [4]:
categories = {0:'Request Water',#Palm
              1:'Request Doctor',#L
              2:'Request Nurse',#Fist
              3:'Request Stroll',#Fist_Moved
              4:'Request PT',#Thumb
              5:'Request CS',#Index
              6:'Request Food',#Ok
              7:'Request Meeting',#Palmmmoved
              8:'Request WC',#C
              9:'Request Help'}#Down

In [5]:
def prepare(filepath):
    IMAGE_SIZE = 200
    image = cv2.cvtColor(filepath, cv2.COLOR_BGR2GRAY)
    new_array = cv2.resize(image,(IMAGE_SIZE,IMAGE_SIZE))
    return new_array.reshape(-1,IMAGE_SIZE,IMAGE_SIZE,1)

In [9]:
import numpy as np
import cv2
import math
capture = cv2.VideoCapture(1)
while capture.isOpened():

    ret, frame = capture.read()
    cv2.rectangle(frame, (100, 100), (250, 250), (0, 255, 0), 0)
    image = frame[100:250, 100:250]
    im_bg = cv2.dilate(image, np.ones((5,5), dtype=np.uint8))
    image_bg = cv2.GaussianBlur(im_bg, (5,5), 1)
    image_bgg = 255 - cv2.absdiff(image, image_bg)
    font = cv2.FONT_HERSHEY_SIMPLEX

   
    _,final_image = cv2.threshold(image_bgg, 240, 255, cv2.THRESH_BINARY_INV)
    

    prediction = loaded_model.predict([prepare(final_image)])

    for i in range(0,10):
        if int(prediction[0][i]) == 1:
            cv2.putText(frame,str(categories[i]),(10,50), font, 2,(255,255,255),2,cv2.LINE_AA)
    
    
    cv2.imshow("Frame", final_image)

    cv2.imshow("Gesture", frame)
    
    if cv2.waitKey(1) == ord('q'):
        break

capture.release()
cv2.destroyAllWindows()